# scBasset on PBMC dataset described at scBasset GitHub page

In [ ]:
import os
import numpy as np
import h5py
import gc
import psutil
import anndata
from scipy import sparse
import tensorflow as tf

# see ig GPU is available
tf.config.list_physical_devices('GPU')

2023-04-06 16:24:42.303617: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2023-04-06 16:24:42.303641: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-06 16:24:59.352523: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[]

2023-04-06 16:24:59.362549: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2023-04-06 16:24:59.370928: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2023-04-06 16:24:59.379255: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2023-04-06 16:24:59.387456

latform/default/dso_loader.cc:64] Could not load dynamic library 'libcusparse.so.11'; dlerror: libcusparse.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2023-04-06 16:24:59.417726: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2023-04-06 16:24:59.417736: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


# Dataset preparation

In [ ]:
"""Run this to create the scBasset input files for the PBMC dataset.
%%bash
source activate scbasset
python /cellar/users/aklie/opt/ml4gland/scBasset/bin/scbasset_preprocess.py \
    --ad_file /cellar/users/aklie/data/ml4gland/use_cases/yuan22/github_tutorial/multiome_pbmc/atac_ad.h5ad \
    --input_fasta /cellar/users/aklie/data/ml4gland/genomes/hg38/hg38.fa \
    --out_path /cellar/users/aklie/data/ml4gland/use_cases/yuan22/github_tutorial/multiome_pbmc/processed \
"""

'Run this to create the scBasset input files for the PBMC dataset.\n%%bash\nsource activate scbasset\npython /cellar/users/aklie/opt/ml4gland/scBasset/bin/scbasset_preprocess.py     --ad_file /cellar/users/aklie/data/ml4gland/use_cases/yuan22/github_tutorial/multiome_pbmc/atac_ad.h5ad     --input_fasta /cellar/users/aklie/data/ml4gland/genomes/hg38/hg38.fa     --out_path /cellar/users/aklie/data/ml4gland/use_cases/yuan22/github_tutorial/multiome_pbmc/processed '

In [ ]:
data_dir = '/cellar/users/aklie/data/ml4gland/use_cases/yuan22/github_tutorial/multiome_pbmc'
split_file = os.path.join(data_dir, 'processed', 'splits.h5')
train_file = os.path.join(data_dir, 'processed', 'train_seqs.h5')
val_file = os.path.join(data_dir, 'processed', 'val_seqs.h5')
test_file = os.path.join(data_dir, 'processed', 'test_seqs.h5')
ad_file = os.path.join(data_dir, 'atac_ad.h5ad')

In [ ]:
# a generator to read examples from h5 file
# create a tf dataset
class generator:
    def __init__(self, file, m):
        self.file = file # h5 file for sequence
        self.m = m # csr matrix, rows as seqs, cols are cells
        self.n_cells = m.shape[1]
        self.ones = np.ones(1344)
        self.rows = np.arange(1344)

    def __call__(self):
        with h5py.File(self.file, 'r') as hf:
            X = hf['X']
            for i in range(X.shape[0]):
                x = X[i]
                x_tf = sparse.coo_matrix((self.ones, (self.rows, x)), 
                                               shape=(1344, 4), 
                                               dtype='int8').toarray()
                y = self.m.indices[self.m.indptr[i]:self.m.indptr[i+1]]
                y_tf = np.zeros(self.n_cells, dtype='int8')
                y_tf[y] = 1
                yield x_tf, y_tf

def print_memory():
    process = psutil.Process(os.getpid())
    print('cpu memory used: %.1fGB.'%(process.memory_info().rss/1e9))

In [ ]:
# Grab the sparse matrix from the anndata object
adata = anndata.read_h5ad(ad_file)
n_cells = adata.shape[0]
m = adata.X.tocoo().transpose().tocsr()

In [ ]:
print_memory()     # memory usage
del adata
gc.collect()

cpu memory used: 0.7GB.


120

In [ ]:
# Get the splits
with h5py.File(split_file, 'r') as hf:
    train_ids = hf['train_ids'][:]
    val_ids = hf['val_ids'][:]

In [ ]:
# Split into train and val
m_train = m[train_ids,:]
m_val = m[val_ids,:]
del m
gc.collect()
m_train.shape, m_val.shape

((24436, 2714), (1357, 2714))

In [ ]:
# Create the tf datasets
train_ds = tf.data.Dataset.from_generator(
        generator(train_file, m_train), 
        output_signature=(
             tf.TensorSpec(shape=(1344,4), dtype=tf.int8),
             tf.TensorSpec(shape=(n_cells), dtype=tf.int8),
        )
    ).shuffle(2000, reshuffle_each_iteration=True).batch(128).prefetch(tf.data.AUTOTUNE)

In [ ]:
val_ds = tf.data.Dataset.from_generator(
        generator(val_file, m_val), 
        output_signature=(
             tf.TensorSpec(shape=(1344,4), dtype=tf.int8),
             tf.TensorSpec(shape=(n_cells), dtype=tf.int8),
        )
    ).batch(128).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Get an example batch from training dataset
for x, y in train_ds.take(1):
    print(x.shape, y.shape)

(128, 1344, 4) (128, 2714)


# Build an scBasset model with their code

In [ ]:
from scbasset.utils import make_model

In [ ]:
model = make_model(32, n_cells)

2023-04-06 16:14:11.749601: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence (InputLayer)          [(None, 1344, 4)]    0           []                               
                                                                                                  
 stochastic_reverse_complement   ((None, 1344, 4),   0           ['sequence[0][0]']               
 (StochasticReverseComplement)   ())                                                              
                                                                                                  
 stochastic_shift (StochasticSh  (None, 1344, 4)     0           ['stochastic_reverse_complement[0
 ift)                                                            ][0]']                           
                                                                                              

In [ ]:
# Set up loss, optimizer, and compile the mdodel
loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.95,beta_2=0.9995)
model.compile(loss=loss_fn, optimizer=optimizer,
                metrics=[tf.keras.metrics.AUC(curve='ROC', multi_label=True),
                        tf.keras.metrics.AUC(curve='PR', multi_label=True)])


In [ ]:
out_dir = os.path.join(data_dir, 'model')

In [ ]:
from datetime import datetime
# earlystopping, track train AUC
filepath = '%s/best_model.h5'%out_dir
    
# tensorboard
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
callbacks = [
        tf.keras.callbacks.TensorBoard(out_dir),
        tf.keras.callbacks.ModelCheckpoint(filepath, save_best_only=True, 
                                           save_weights_only=True, monitor='auc', mode='max'),
        tf.keras.callbacks.EarlyStopping(monitor='auc', min_delta=1e-6, 
                                         mode='max', patience=50, verbose=1),
    ]

In [ ]:
import pickle

In [ ]:
history = model.fit(
        train_ds,
        epochs=1000,
        callbacks=callbacks,
        validation_data=val_ds)
pickle.dump(history.history, open('%s/history.pickle'%out_dir, 'wb'))